In [2]:
import os
from os import path
import sqlite3
global_sqlite_filename='../apf.db'
import pandas

In [3]:
class col_types:
    def __init__(self):
        self.TEXT = 'TEXT'
        self.INTEGER = 'INTEGER'
        self.REAL = 'REAL'
        self.BLOB = 'BLOB'
        self.NULL = 'NULL'
        
ct = col_types()
print(ct.TEXT)

print(os.getcwd())
print(os.listdir())

TEXT
/Volumes/AARONTHUMB1/APF_telescope_behavior_study
['.git', '.ipynb_checkpoints', 'Create_and_populate_database.ipynb', 'add_telemetry_files.sh', 'basic_glance_of_data .ipynb', 'create_telemetry_table.sh', 'delete_telemetry_table.sh', 'import.txt', 'populate_telemetry_table.sh', 'tiny_subset_data']


In [4]:
def create_database(sqlite_filename=global_sqlite_filename):
    conn = sqlite3.connect(sqlite_filename)
    cursor = conn.cursor()
    return cursor
       
def get_telemetry_columns():
    columns = {'DateTime': ct.TEXT, 
                  'TARGET': ct.NULL,
                  'MIDPTFIN': ct.REAL,
                  'AZ': ct.REAL,
                  'EL': ct.REAL,
                  'AZENCPOS': ct.REAL,
                  'ELENCPOS': ct.REAL,
                  'AZENCVEL': ct.REAL,
                  'ELENCVEL': ct.REAL,
                  'AZFLWERR': ct.REAL,
                  'ELFLWERR': ct.REAL,
                  'OUTFILE': ct.REAL,
                  'OBSNUM': ct.REAL,
                  'MODE': ct.REAL,
                  'AVG_FWHM': ct.REAL,
                  'M5WIND': ct.REAL,
                  'M5WINDAZ': ct.REAL,
                  'TAVERAGE': ct.REAL,
                  'TM1S210': ct.REAL,
                  'TM2CAIR': ct.REAL,
                  'OFFSET_AZ': ct.REAL,
                  'OFFSET_EL': ct.REAL,
                  'RMSOFFSET_AZ': ct.REAL,
                  'RMSOFFSET_EL': ct.REAL,
                  'AVGOFFSET_AZ': ct.REAL,
                  'AVGOFFSET_EL': ct.REAL,
                  'HATCHPOS': ct.TEXT,
                  'EVENT': ct.TEXT}    
    return columns
    
def create_telemetry_table(cursor):        
    columns_table1 = get_telemetry_columns()
    l = ['{} {}'.format(k,v) for (k, v) in columns_table1.items()]
    print(len(l))
    full_list = ', '.join(l)
    sql_create_string = 'CREATE TABLE telemetry (' + full_list + ', PRIMARY KEY (DateTime))'
    print(sql_create_string)        
        
    # Creating a new SQLite table 
    cursor.execute('drop table if exists telemetry')
    cursor.execute(sql_create_string)
    
    return

def open_connection(sqlite_filename=global_sqlite_filename):
    dburi = 'file:{}?mode=rw'.format(sqlite_filename)
    conn = sqlite3.connect(dburi, uri=True)
    return conn
    

In [ ]:
cursor = create_database()
create_telemetry_table(cursor)

28
CREATE TABLE telemetry (DateTime TEXT, TARGET NULL, MIDPTFIN REAL, AZ REAL, EL REAL, AZENCPOS REAL, ELENCPOS REAL, AZENCVEL REAL, ELENCVEL REAL, AZFLWERR REAL, ELFLWERR REAL, OUTFILE REAL, OBSNUM REAL, MODE REAL, AVG_FWHM REAL, M5WIND REAL, M5WINDAZ REAL, TAVERAGE REAL, TM1S210 REAL, TM2CAIR REAL, OFFSET_AZ REAL, OFFSET_EL REAL, RMSOFFSET_AZ REAL, RMSOFFSET_EL REAL, AVGOFFSET_AZ REAL, AVGOFFSET_EL REAL, HATCHPOS TEXT, EVENT TEXT, PRIMARY KEY (DateTime))


In [6]:
conn = open_connection()
cursor = conn.cursor()

In [7]:
telemetry_columns = get_telemetry_columns()
print([k for (k, v) in telemetry_columns.items()])

['DateTime', 'TARGET', 'MIDPTFIN', 'AZ', 'EL', 'AZENCPOS', 'ELENCPOS', 'AZENCVEL', 'ELENCVEL', 'AZFLWERR', 'ELFLWERR', 'OUTFILE', 'OBSNUM', 'MODE', 'AVG_FWHM', 'M5WIND', 'M5WINDAZ', 'TAVERAGE', 'TM1S210', 'TM2CAIR', 'OFFSET_AZ', 'OFFSET_EL', 'RMSOFFSET_AZ', 'RMSOFFSET_EL', 'AVGOFFSET_AZ', 'AVGOFFSET_EL', 'HATCHPOS', 'EVENT']


In [ ]:
#telemetry1df = pandas.read_csv('tiny_subset_data/subset_2015_01_01_02_offsets', 
                               sep='\t', skiprows=0, header=None, names=telemetry_columns)
#print(telemetry1df)
#telemetry1df.to_sql('telemetry', conn, if_exists='append', index=False)


In [ ]:
#telemetrybig = pandas.read_csv('../telemetry_by_year/2015/2015_01_01_02_offsets', 
 #                              sep='\t', skiprows=1, header=None, names=telemetry_columns)

In [8]:
def populate_telemetry_table(conn, filenames):
    n=0
    for csv_file in filenames:
        n = n+1
        try:
            df = pandas.read_csv(csv_file, 
                               sep='\t', skiprows=1, header=None, names=get_telemetry_columns(), comment='#')
            df.to_sql('telemetry', conn, if_exists='append', index=False)
            
        except Exception as e:
            print(csv_file)
            print(e)

def populate_telemetry(conn=None, dirname=None, batchsize=5):
    filenames = [os.path.join(dirname, x) for x in os.listdir(dirname)]
    filenames.sort()
    
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx : min(ndx + n, l)]
    
    n=0
    for file_batch in batch(filenames, n=batchsize):
        populate_telemetry_table(conn, file_batch)
        n += batchsize
        print('{}: {}'.format(n, file_batch[0]))
    


        

In [9]:
populate_telemetry(conn, '../telemetry_data', batchsize=10)

/Users/aaron/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


20: ../telemetry_data/2015_01_01_02_offsets
../telemetry_data/2015_01_29_30_offsets
UNIQUE constraint failed: telemetry.DateTime
../telemetry_data/2015_02_04_05_offsets
UNIQUE constraint failed: telemetry.DateTime
40: ../telemetry_data/2015_01_23_24_offsets


KeyboardInterrupt: 